In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('NYC_Taxi_dataset_with_anomalies.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance_miles,fare_amount,passenger_count,payment_type
0,2023-02-20 17:27:00,2023-02-20 17:49:00,40.808941,-73.914482,40.807336,-73.905270,1.03,5.84$,5.0,Credit Card
1,2023-02-28 19:41:00,2023-02-28 20:07:00,40.685842,-73.855449,40.663358,-73.826745,4.02,15.6£,4.0,Unknown
2,2023-02-14 08:37:00,2023-02-14 09:07:00,40.668055,-74.045050,40.642572,-74.055617,3.04,2192.65¥,5.0,Credit Card
3,2023-01-16 23:54:00,2023-01-17 00:40:00,40.765394,-73.753324,40.775285,-73.775441,2.67,0,4.0,Credit Card
4,2023-02-11 01:16:00,2023-02-11 01:35:00,40.815841,-73.727328,40.836115,-73.746976,3.11,13.7€,3.0,Credit Card


In [11]:
dfbis = pd.read_csv('NYC_Taxi_dataset_with_anomalies_bis.csv')
dfbis = dfbis.drop(columns=['Unnamed: 0'])
dfbis.head()

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance_miles,fare_amount,passenger_count,payment_type
0,2023-02-20 17:27:00,2023-02-20 17:49:00,40.808941,-73.914482,40.807336,-73.905270,1.03,5.84$,5.0,Credit Card
1,2023-02-28 19:41:00,2023-02-28 20:07:00,40.685842,-73.855449,40.663358,-73.826745,4.02,15.6£,4.0,Unknown
2,2023-02-14 08:37:00,2023-02-14 09:07:00,40.668055,-74.045050,40.642572,-74.055617,3.04,2192.65¥,5.0,Credit Card
3,2023-01-16 23:54:00,2023-01-17 00:40:00,40.765394,-73.753324,40.775285,-73.775441,2.67,0,4.0,Credit Card
4,2023-02-11 01:16:00,2023-02-11 01:35:00,40.815841,-73.727328,40.836115,-73.746976,3.11,13.7€,3.0,Credit Card


In [12]:
# parsing des dates
dfbis['pickup_datetime'] = pd.to_datetime(dfbis['pickup_datetime'], errors='coerce')
dfbis['dropoff_datetime'] = pd.to_datetime(dfbis['dropoff_datetime'], errors='coerce')

In [13]:
#date inverser

dfbis[dfbis['pickup_datetime'] > dfbis['dropoff_datetime']]


,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance_miles,fare_amount,passenger_count,payment_type


### Gestion de null dnas les colonnes

In [14]:
df.isna().sum()

pickup_datetime        42
dropoff_datetime        0
pickup_latitude         0
pickup_longitude        0
dropoff_latitude        0
dropoff_longitude       0
trip_distance_miles    40
fare_amount            38
passenger_count        40
payment_type            0
dtype: int64

pickup_datetime à notre avis, est un attribut critique. Pour les lignes sans pickup_datetime, on n'a pas de pas de durée fiable, pas de vitesse, pas d’analyse temporelle possible. On a plutot une ambiguïté métier. 

In [15]:
df = df.dropna(subset=['pickup_datetime']).copy()

Pour la colonne ``trip_distance_miles``, nous allons imputer les valeurs manquantes en utilisant la médiane de la vitesse calculée à partir des temps de trajet et distance disponibles. Ensuite faire le produit croisé avec le temps de trajet pour estimer la distance.

In [16]:
#parsing des dates
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], errors='coerce')
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], errors='coerce')

# calculer la vitesse médiane (sur données valides)
valid_mask = (
    df['trip_distance_miles'].notna() &
    df['pickup_datetime'].notna() &
    df['dropoff_datetime'].notna()
)

df['trip_duration_h'] = (
    df['dropoff_datetime'] - df['pickup_datetime']
).dt.total_seconds() / 3600

median_speed_mph = (
    df.loc[valid_mask, 'trip_distance_miles'] /
    df.loc[valid_mask, 'trip_duration_h']
).median()

In [17]:
#imputer la distance manquante
mask_nan = df['trip_distance_miles'].isna() & df['trip_duration_h'].notna()

df.loc[mask_nan, 'trip_distance_miles'] = (
    df.loc[mask_nan, 'trip_duration_h'] * median_speed_mph
)

Pour l'imputationd de ``passenger_count`` , nous allons imputer par le mode

In [18]:
passenger_mode = df['passenger_count'].mode().iloc[0]
df['passenger_count'] = df['passenger_count'].fillna(passenger_mode)

In [19]:
df = df[["pickup_datetime", "dropoff_datetime", "pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude", "trip_distance_miles", "fare_amount", "passenger_count", "payment_type"]]
df.isna().sum()

pickup_datetime         0
dropoff_datetime        0
pickup_latitude         0
pickup_longitude        0
dropoff_latitude        0
dropoff_longitude       0
trip_distance_miles     0
fare_amount            35
passenger_count         0
payment_type            0
dtype: int64

En ce qui concerne l'attribut ``fare_amount``, le type de la colonne est string et contient des caractère spéciaux, nous allons l'imputer une fois ses valeurs numériques extraites et standardisées plus bas

#### suppression des lignes dupliquées

In [20]:
#les duplicatas
df.duplicated().sum()

np.int64(10)

In [21]:
#suppression des duplicatas
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

In [22]:
df_to_improve = df.copy()


In [23]:
df_to_improve.dtypes

pickup_datetime        datetime64[ns]
dropoff_datetime       datetime64[ns]
pickup_latitude               float64
pickup_longitude              float64
dropoff_latitude              float64
dropoff_longitude             float64
trip_distance_miles           float64
fare_amount                    object
passenger_count               float64
payment_type                   object
dtype: object

In [24]:
# etat date dans le futur
now = pd.Timestamp.now()

report = {
        "pickup_in_future": int((df_to_improve['pickup_datetime'] > now).sum()),
        "dropoff_in_future": int((df_to_improve['dropoff_datetime'] > now).sum()),
    }
report

{'pickup_in_future': 0, 'dropoff_in_future': 0}

On peut constater que nos 40 lignes de dropoff dans le future (detecté dnas l'étape 3) ont été supprimees avec les gestion de valeurs manquantes.

Si c'est 40 lignes étaient toujours présentes sur 800 lignes au total, on aurait environ ~5 % avec des dates dans le futur, c’est assez peu dans notre contexte.

En ce moment nous procédérons tout simplement à la suppresion de ces lignes. Comme ce qui suit 

In [25]:
now = pd.Timestamp.now()

df_clean_without_future_date = df_to_improve[
    (df_to_improve['pickup_datetime'] <= now) &
    (df_to_improve['dropoff_datetime'] <= now)
].copy()

In [26]:
# lignes où dropoff est avant pickup
inverted_dates = df_clean_without_future_date[
    df_clean_without_future_date['dropoff_datetime'] < df_clean_without_future_date['pickup_datetime']
]
inverted_dates.shape[0]

0

On n'a pas de date pickup > dropoff

####  Normalize fares -> fare_amount_usd


In [27]:
def convert_to_usd(fare) -> float:
    """Robust single-value conversion: string with currency symbol or numeric -> USD (float) or np.nan."""
    try:
        if pd.isna(fare):
            return np.nan
        if isinstance(fare, (int, float, np.number)):
            return float(fare)
        fare_str = str(fare).strip()
        if fare_str == '':
            return np.nan
        # currency detection
        if '€' in fare_str:
            return float(fare_str.replace('€', '').strip()) * 1.18
        if '£' in fare_str:
            return float(fare_str.replace('£', '').strip()) * 1.33
        if '¥' in fare_str or 'JPY' in fare_str:
            return float(fare_str.replace('¥', '').replace('JPY', '').strip()) * 0.009
        if '$' in fare_str:
            return float(fare_str.replace('$', '').strip())
        # fallback numeric
        return round(float(fare_str), 2)
    except Exception:
        return np.nan


def normalize_fares(df, col = "fare_amount", out_col = "fare_amount_usd"):
    """Create numeric USD fare column and report conversion stats."""
    df = df.copy()
    df[out_col] = df[col].apply(convert_to_usd)
    
    report = {
        "total": len(df),
        "converted_notnull": int(df[out_col].notna().sum()),
        "converted_null": int(df[out_col].isna().sum()),
        "null ou non positif": int((df[out_col] <= 0).sum())
    }
    return df, report

In [28]:
work_df, rep_fares = normalize_fares(df_clean_without_future_date, col="fare_amount", out_col="fare_amount_usd")
rep_fares

{'total': 772,
 'converted_notnull': 737,
 'converted_null': 35,
 'null ou non positif': 23}

Sur l'attribut ``fare_amount_usd`` on a 35 valeurs NAN et 23 valeurs null. Ce qui represente environ 7,5% des valeurs de la colonne.

Pour l'impution nous allons considerer le tarif median

In [29]:
#  la médiane sur les valeurs valides
median_fare = work_df.loc[work_df['fare_amount_usd'] > 0, 'fare_amount_usd'].median()

# imputer NaN et valeurs ≤ 0
mask_invalid = (work_df['fare_amount_usd'].isna()) | (work_df['fare_amount_usd'] <= 0)

work_df.loc[mask_invalid, 'fare_amount_usd'] = median_fare

In [30]:
work_df.drop(columns=['fare_amount'], inplace=True)
work_df.isna().sum()

pickup_datetime        0
dropoff_datetime       0
pickup_latitude        0
pickup_longitude       0
dropoff_latitude       0
dropoff_longitude      0
trip_distance_miles    0
passenger_count        0
payment_type           0
fare_amount_usd        0
dtype: int64

#### Gestion des valeurs abérantes constatées dans l'étapes 3

On avait noter selon la méthode IQR que 89.68% des valeurs se trouvait dans l'intervalle de valeur acceptable (avant biensur les suppressions de lignes faites plus haut).
Pour donc gerer les valeurs aberantes ici qui sont >10%, nous allons procédé à limputation par la valeur médianne 

In [31]:
# Les valeurs seuil 

Q1 = work_df['fare_amount_usd'].quantile(0.25)
Q3 = work_df['fare_amount_usd'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# impute par la valeur medianne
median_fare = work_df['fare_amount_usd'].median()

outliers = work_df[
    (work_df['fare_amount_usd'] < lower_bound) | 
    (work_df['fare_amount_usd'] > upper_bound)
]

print(f"Nombre d'outliers avant : {len(outliers)}")

# Remplacer les outliers par la médiane
work_df['fare_amount_usd'] = work_df['fare_amount_usd'].mask(
    (work_df['fare_amount_usd'] < lower_bound) | (work_df['fare_amount_usd'] > upper_bound),
    median_fare
)

outliers = work_df[
    (work_df['fare_amount_usd'] < lower_bound) | 
    (work_df['fare_amount_usd'] > upper_bound)
]

print(f"Nombre d'outliers après : {len(outliers)}")


Nombre d'outliers avant : 36
Nombre d'outliers après : 0


On constate que la suppresion des lignes a fait baissé le nombre de doutliers de fare_amount_usd dans nos données. L'imputation ici par la valeur médianne reste acceptable.

#### Gestion des coordonnées hors NYC
Dans l'étape 03, on avait pu constater la présence de coordonnées géographiques hors de NYC


In [32]:
# Approximation coordonnées ville de new York
nyc_min_latitude = 40.5
nyc_max_latitude = 40.92   
nyc_min_longitude = -74.26 
nyc_max_longitude = -73.7
invalid_pickup = work_df[
    (work_df['pickup_latitude'] < nyc_min_latitude) | (work_df['pickup_latitude'] > nyc_max_latitude) |
    (work_df['pickup_longitude'] < nyc_min_longitude) | (work_df['pickup_longitude'] > nyc_max_longitude)
]
invalid_pickup.shape[0]
invalid_dropoff = work_df[
    (work_df['dropoff_latitude'] < nyc_min_latitude) | (work_df['dropoff_latitude'] > nyc_max_latitude) |
    (work_df['dropoff_longitude'] < nyc_min_longitude) | (work_df['dropoff_longitude'] > nyc_max_longitude)
]
invalid_dropoff.shape[0]

print(f"nombre de coordonnées(pickup ou dropoff) invalides: {invalid_pickup.shape[0] + invalid_dropoff.shape[0]} -- total lignes: {work_df.shape[0]}")

nombre de coordonnées(pickup ou dropoff) invalides: 95 -- total lignes: 772


On a environ 10% des valeurs qui ne sont pas à NYC pour (pickup et dropoff compris),

Pour l'imputation nous allons remplacer les valeurs par les coordonnées mode

In [33]:
# on remplace les coordonnées invalides par le mode des coordonnées valides

# on recupere le mode des coordonnees valides pour pcickup et impute les invalides
valid_pickup_lat_mode = work_df.loc[
    (work_df['pickup_latitude'] >= nyc_min_latitude) & (work_df['pickup_latitude'] <= nyc_max_latitude),
    'pickup_latitude'
].mode()[0]
work_df['pickup_latitude'] = work_df['pickup_latitude'].apply(
    lambda x: valid_pickup_lat_mode if (x < nyc_min_latitude or x > nyc_max_latitude) else x
)

# on recupere le mode des coordonnees valides pour pcickup et impute les invalides
valid_pickup_lon_mode = work_df.loc[
    (work_df['pickup_longitude'] >= nyc_min_longitude) & (work_df['pickup_longitude'] <= nyc_max_longitude),
    'pickup_longitude'
].mode()[0]
work_df['pickup_longitude'] = work_df['pickup_longitude'].apply(
    lambda x: valid_pickup_lon_mode if (x < nyc_min_longitude or x > nyc_max_longitude) else x
)

# on recupere le mode des coordonnees valides pour dropoff et impute les invalides
valid_dropoff_lat_mode = work_df.loc[
    (work_df['dropoff_latitude'] >= nyc_min_latitude) & (work_df['dropoff_latitude'] <= nyc_max_latitude),
    'dropoff_latitude'
].mode()[0]
work_df['dropoff_latitude'] = work_df['dropoff_latitude'].apply(
    lambda x: valid_dropoff_lat_mode if (x < nyc_min_latitude or x > nyc_max_latitude) else x
)

# on recupere le mode des coordonnees valides pour dropoff et impute les invalides
valid_dropoff_lon_mode = work_df.loc[
    (work_df['dropoff_longitude'] >= nyc_min_longitude) & (work_df['dropoff_longitude'] <= nyc_max_longitude),
    'dropoff_longitude'
].mode()[0]
work_df['dropoff_longitude'] = work_df['dropoff_longitude'].apply(
    lambda x: valid_dropoff_lon_mode if (x < nyc_min_longitude or x > nyc_max_longitude) else x
)

# re-evaluation des coordonnées invalides
invalid_pickup = work_df[
    (work_df['pickup_latitude'] < nyc_min_latitude) | (work_df['pickup_latitude'] > nyc_max_latitude) |
    (work_df['pickup_longitude'] < nyc_min_longitude) | (work_df['pickup_longitude'] > nyc_max_longitude)
]
invalid_pickup.shape[0]
invalid_dropoff = work_df[
    (work_df['dropoff_latitude'] < nyc_min_latitude) | (work_df['dropoff_latitude'] > nyc_max_latitude) |
    (work_df['dropoff_longitude'] < nyc_min_longitude) | (work_df['dropoff_longitude'] > nyc_max_longitude)
]   
invalid_dropoff.shape[0]
print(f"nombre de coordonnées(pickup ou dropoff) invalides après imputation: {invalid_pickup.shape[0] + invalid_dropoff.shape[0]} -- total lignes: {work_df.shape[0]}") 



nombre de coordonnées(pickup ou dropoff) invalides après imputation: 0 -- total lignes: 772


Toutes les valeurs (coordonnées gps) invalides ont été imputé

#### Payment_type

En faisant le data profiling du dataset à l'étape 3 on avait pu constater la présence de valeurs avec la même sémantique mais orthographié, ou exprimé par des string différent.

In [34]:
work_df['payment_type'].unique()


array(['Credit Card', 'Unknown', 'Cash', ' CREDIT CARD ', 'credit card',
       'cash', 'Dispute', 'No Charge', 'CREDIT CARD', 'Disagreement'],
      dtype=object)

In [35]:
#Uniformiser les valeurs
work_df['payment_type'] = work_df['payment_type'].str.strip().str.lower()

#  regroupons
work_df['payment_type'] = work_df['payment_type'].replace({
    'credit card': 'credit_card',
    'cash': 'cash',
    'unknown': 'unknown',
    'dispute': 'dispute',
    'no charge': 'no_charge',
    'disagreement': 'dispute'  # Exemple de regroupement
})

work_df['payment_type'].unique()


array(['credit_card', 'unknown', 'cash', 'dispute', 'no_charge'],
      dtype=object)

### Gestion valeurs aberant sur le tuples ('pickup_datetime', 'dropoff_datetime', 'trip_distance_miles')

Dans l'étape de l'assessmen, on avait trouvé des valeurs aberantes en calculant la vitesse moyenne de certains trajet. Des vitesses qui allait jusqu'à 120 mph (soit près de 200km/h)

On avait donc fixé comme seuil de vitesse min = 0.5 mph (circulation dense, embouteillage) et
max = 60 mph (environ 100km/h)

In [36]:
# sur la vitesse en utilisant les champs pickup_datetime, dropoff_datetime et trip_distance_miles
def calculate_speed(row):
    try:
        pickup_time = pd.to_datetime(row['pickup_datetime'])
        dropoff_time = pd.to_datetime(row['dropoff_datetime'])
        trip_distance = row['trip_distance_miles']
        
        if pd.isna(pickup_time) or pd.isna(dropoff_time) or trip_distance <= 0:
            return np.nan
        
        trip_duration = (dropoff_time - pickup_time).total_seconds() / 3600  # durée en heures
        if trip_duration <= 0:
            return np.nan
        
        speed = trip_distance / trip_duration  # vitesse en miles par heure
        return speed
    except Exception as e:
        return np.nan
    
work_df['speed_mph'] = work_df.apply(calculate_speed, axis=1)



In [37]:
min_speed_threshold = 0.5  # mph
max_speed_threshold = 60  # mph

invalid_speed = work_df[
    (work_df['speed_mph'] < min_speed_threshold) | (work_df['speed_mph'] > max_speed_threshold)
]
print(f"Nombre de trajets avec vitesse invalide: {len(invalid_speed)} sur un total de {len(work_df)} trajets.")

Nombre de trajets avec vitesse invalide: 19 sur un total de 772 trajets.


On a environ 2% des trajets qui ont des vitesses invalides. Pour 2% nous allons tout simplement laisser tomber ces lignes

In [38]:
#filtrer les lignes avec trajet 
df_improve = work_df[
    (work_df['speed_mph'] >= min_speed_threshold) & (work_df['speed_mph'] <= max_speed_threshold)
]
df_improve.shape[0]

753

In [39]:
df_improved = df_improve[["pickup_datetime", "dropoff_datetime", "pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude", "trip_distance_miles", "fare_amount_usd", "passenger_count", "payment_type"]]
df_improved.shape

(753, 10)

In [40]:
df_improved.to_csv('NYC_Taxi_dataset_improved.csv', index=False)